# **Plant Leaf Disease Detection Model Training**

### Step1: Importing Neccesary Libraries

In [ ]:
import os, gc
import cv2, PIL

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from keras import models
from keras.applications import MobileNetV2

from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import shutil

### Step2: Defining Model Training Class

In [ ]:
class ModelTrain:
    es=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=7,restore_best_weights=True,start_from_epoch=3)
    main_dir=""
    def __init__(self,main_dir):
        self.main_dir=main_dir
        
    ## ImageDataGenerator to apply Data Augmentation
    def imgDataGen(self,rotation=0,zoom=1,hflip=False,vflip=False):
        train_datagen = keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            rotation_range=rotation,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.15,
            zoom_range=zoom,
            horizontal_flip=hflip,
            vertical_flip=vflip,
            fill_mode='nearest',    
        )
        test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
        valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
        return train_datagen,test_datagen,valid_datagen
    
    ## Loading images from the Directory structure
    def flowImgData(self,folder,datagen,image_shape=(224,224),class_mode='sparse'):
        generator = datagen.flow_from_directory(
            os.path.join(self.main_dir,folder),
            target_size=image_shape,
            batch_size=32,
            class_mode=class_mode,
            shuffle=True
        )
        print("Generator Class Indices : ",generator.class_indices)
        print("Number of Classes : ",len(generator.class_indices))
        return generator
    
    ## Model Training based on given Parameters
    def trainModel(self,crop_name,train_generator,valid_generator,model,cw,save_path="",epoch=50,early_stoping=es):     
        history = model.fit(
              train_generator,
              steps_per_epoch=train_generator.samples/train_generator.batch_size ,
              epochs=epoch,
              validation_data=valid_generator,
              validation_steps=valid_generator.samples/valid_generator.batch_size,
              verbose=1,
            class_weight=cw,
            callbacks=[self.es],
        )

        if(save_path!=""):
            model.save(save_path)
            shutil.make_archive(crop_name+"_mobileNet5",'zip', save_path)
        return history,model
    
    ## Model Performace Visualization
    def plotHistory(self,history,crop):
        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        epochs = range(len(acc))
        plt.plot(epochs, acc, 'r', label='Training accuracy')
        plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
        plt.title(f'{crop}: Training and validation accuracy')
        plt.legend(loc=0)
        plt.figure()
        plt.show()
        plt.savefig(f"/kaggle/working/models/{crop}_acc_vs_val_acc.png")
        plt.plot(epochs, loss, 'r', label='Training Loss')
        plt.plot(epochs, val_loss, 'b', label='Validation Loss')
        plt.title(f'{crop}: Training and validation loss')
        plt.legend(loc=0)
        plt.figure()
        plt.show()
        plt.savefig(f"/kaggle/working/models/{crop}_loss_vs_val_loss.png")
    
    ## Predicting Image Class
    def predict(self,model,img_path,image_shape=(224,224)):
        img = keras.preprocessing.image.load_img(
            img_path, target_size=image_shape
        )
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)
        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        confidence=np.argmax(score)
        return predictions,confidence
    
    ## Performance on test Data
    def testAccuracy(self,test_generator,model):
        test_loss, test_acc = model.evaluate(test_generator, verbose=2)
        print("Test Accuracy : ",test_acc)
        return test_acc
    
    def predictAll(self,model,dir):
        predictionArray=[]
        confidenceArray=[]
        for i in os.listdir(dir):
            img_path=os.path.join(dir,i)
            pred,confidence=self.predict(model,img_path)
            predictionArray.append(pred)
            confidenceArray.append(confidence)
            print("Prediction : ",pred)
            print("Confidence : ",confidence)
            print("Image Path : ",img_path)
            print("Class : ",i)
            print("--------------------------------------------------------")
        predictionArray=np.array(predictionArray)
        confidenceArray=np.array(confidenceArray)
        return predictionArray,confidenceArray
    

In [ ]:

data_dir="/kaggle/input/individual-crop-leaf-disease-dataset/Individual_crop_leaf_disease_data/"
crop_count=0;
history_list=[]
model_list=[]
for i in sorted(os.listdir(data_dir))[0:1]:
    print("---------------------------------------------")
    print("Crop Name : ",i)
    print("---------------------------------------------")
    main_dir=os.path.join(data_dir,i)
    print("Directory : ",main_dir)
    print("---------------------------------------------")
    modelTrain=ModelTrain(main_dir)
    
    train_datagen,test_datagen,valid_datagen=modelTrain.imgDataGen(rotation=20,zoom=0.15,hflip=True,vflip=True)

    train_generator=modelTrain.flowImgData("train",train_datagen)
    test_generator=modelTrain.flowImgData("test",test_datagen)
    valid_generator=modelTrain.flowImgData("val",valid_datagen)
    
    num_classes=len(train_generator.class_indices)
#####################################
    base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    head_model = base_model.output
    head_model = layers.AveragePooling2D(pool_size=(7, 7))(head_model)
    head_model = layers.Flatten(name="flatten")(head_model)
    head_model = layers.Dense(1024, activation="relu")(head_model)
    head_model = layers.Dropout(0.5)(head_model)
    head_model = layers.Dense(1024, activation="relu")(head_model)
    head_model = layers.Dropout(0.3)(head_model)
    head_model = layers.Dense(512, activation="relu")(head_model)
    head_model = layers.Dropout(0.2)(head_model)
    head_model = layers.Dense(256, activation="relu")(head_model)
    head_model = layers.Dropout(0.1)(head_model)
    head_model = layers.Dense(num_classes, activation="softmax")(head_model)
    model = models.Model(inputs=base_model.input, outputs=head_model)
    
    for layer in base_model.layers:
        layer.trainable = False
#####################################
    from sklearn.utils import class_weight
    class_weights = class_weight.compute_sample_weight(
        'balanced',
        train_generator.classes)
    class_weight_dict = dict(enumerate(class_weights))
    
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizers.Adam(), metrics=["accuracy"])
    history="helo"
    history,model=modelTrain.trainModel(i,train_generator,valid_generator,model=model,save_path=os.path.join("/kaggle/working/models",f"{i}_mobileNet.pb"),cw=class_weight_dict)
    history_list.append(history)
    model_list.append(model)

    modelTrain.plotHistory(history,i)
    modelTrain.testAccuracy(test_generator,model)
    gc.collect()
    shutil.make_archive("models_compressed",'zip',"/kaggle/working/models")
    
    crop_count+=1
    print("--------------------------------------------------------")
    

In [ ]:
import pickle
with open('/kaggle/working/history.txt', 'wb') as h:
    pickle.dump(history_list, h)
with open('/kaggle/working/models.txt', 'wb') as m:
    pickle.dump(model_list, m)

In [ ]:
model.evaluate(test_generator)

In [ ]:
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)